<h1 style="padding-top: 25px;padding-bottom: 25px;text-align: left; padding-left: 10px; background-color: #DDDDDD; 
    color: black;"> <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> AC295: Advanced Practical Data Science </h1>

## Attention and Transformers

**Harvard University, Fall 2020**  
**Instructors**: Pavlos Protopapas  

---

**Each assignment is graded out of 5 points.  The topic for this assignment is Transfer Learning for Text.**

**Due:** 11/03/2020 10:15 AM EDT

**Submit:** We won't be re running your notebooks, please ensure output is visible in the notebook.

#### Learning Objectives

In this exercise you will cover the following topics:  
- Tokenizing text for BERT
- BERT for Text Classification Task

---

#### Installs

In [1]:
!pip install transformers #Installing Huggingface transformers 
!pip install ipdb # for debugging

     |████████████████████████████████| 1.3MB 3.4MB/s 
     |████████████████████████████████| 890kB 12.8MB/s 
     |████████████████████████████████| 1.1MB 24.5MB/s 
     |████████████████████████████████| 2.9MB 31.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=e8e6652b78a6acf0c26ba0228c02fe2eae9eb75236ac95a64717287535094d43
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Created wheel for ipdb: filename=ipdb-0.13.4-cp36-none-any.whl size=10973 sha256=9840eb2c041d6e8324f907b2c794692e3488c26c1a1e3710ab91a8dc1d4135dc
  Stored in directory: /root/.cache/pip/wheels/56/51/e4/c91c61e3481a1a967beb18c4ea7a2b138a63cce94170b2e206
Successfully built ipdb


#### Imports

In [2]:
import os
import requests
import zipfile
import tarfile
import shutil
import json
import time
import sys
import string
import re
import numpy as np
import pandas as pd
from glob import glob
from string import Template
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras import backend as K
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split

from transformers import BertTokenizer, TFBertForSequenceClassification

#### Verify Setup

In [3]:
# Enable/Disable Eager Execution
# Reference: https://www.tensorflow.org/guide/eager
# TensorFlow's eager execution is an imperative programming environment that evaluates operations immediately, 
# without building graphs

#tf.compat.v1.disable_eager_execution()
#tf.compat.v1.enable_eager_execution()

print("tensorflow version", tf.__version__)
print("keras version", tf.keras.__version__)
print("Eager Execution Enabled:", tf.executing_eagerly())

# Get the number of replicas 
strategy = tf.distribute.MirroredStrategy()
print("Number of replicas:", strategy.num_replicas_in_sync)

devices = tf.config.experimental.get_visible_devices()
print("Devices:", devices)
print(tf.config.experimental.list_logical_devices('GPU'))

print("GPU Available: ", tf.config.list_physical_devices('GPU'))
print("All Physical Devices", tf.config.list_physical_devices())

# Better performance with the tf.data API
# Reference: https://www.tensorflow.org/guide/data_performance
AUTOTUNE = tf.data.experimental.AUTOTUNE

tensorflow version 2.3.0
keras version 2.4.0
Eager Execution Enabled: True
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Number of replicas: 1
Devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
[]
GPU Available:  []
All Physical Devices [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU')]


#### Utils

In [4]:
def download_file(packet_url, base_path="", extract=False):
  if base_path != "":
    if not os.path.exists(base_path):
      os.mkdir(base_path)
  packet_file = os.path.basename(packet_url)
  with requests.get(packet_url, stream=True) as r:
      r.raise_for_status()
      with open(os.path.join(base_path,packet_file), 'wb') as f:
          for chunk in r.iter_content(chunk_size=8192):
              f.write(chunk)
  
  if extract:
    if packet_file.endswith(".zip"):
      with zipfile.ZipFile(os.path.join(base_path,packet_file)) as zfile:
        zfile.extractall(base_path)
    
    if packet_file.endswith(".tar.gz"):
      packet_name = packet_file.split('.')[0]
      with tarfile.open(os.path.join(base_path,packet_file)) as tfile:
        tfile.extractall(base_path)

def evaluate_model(model,test_data, training_results):
    
  # Get the model train history
  model_train_history = training_results.history
  # Get the number of epochs the training was run for
  num_epochs = len(model_train_history["loss"])

  # Plot training results
  fig = plt.figure(figsize=(15,5))
  axs = fig.add_subplot(1,2,1)
  axs.set_title('Loss')
  # Plot all metrics
  for metric in ["loss","val_loss"]:
      axs.plot(np.arange(0, num_epochs), model_train_history[metric], label=metric)
  axs.legend()
  
  axs = fig.add_subplot(1,2,2)
  axs.set_title('Accuracy')
  # Plot all metrics
  for metric in ["accuracy","val_accuracy"]:
      axs.plot(np.arange(0, num_epochs), model_train_history[metric], label=metric)
  axs.legend()

  plt.show()
  
  # Evaluate on test data
  evaluation_results = model.evaluate(test_data)
  print("Evaluation Results:", evaluation_results)

## Dataset

**We will continue to use the dataset from Exercise 6.** The dataset consists of news articles from CNN in the politics, health, and entertainment category. There are about 300 articles in each category.

#### Download

In [5]:
start_time = time.time()
download_file("https://storage.googleapis.com/dataset_store/ac295/news300.zip", base_path="datasets", extract=True)
execution_time = (time.time() - start_time)/60.0
print("Download execution time (mins)",execution_time)

Download execution time (mins) 0.006757978598276774


#### Explore

In [6]:
data_dir = os.path.join("datasets","news300")
label_names = os.listdir(data_dir)

# Number of unique labels
num_classes = len(label_names) 
# Create label index for easy lookup
label2index = dict((name, index) for index, name in enumerate(label_names))
index2label = dict((index, name) for index, name in enumerate(label_names))

print("Number of classes:", num_classes)
print("Labels:", label_names)

# Generate a list of labels and path to text
data_x = []
data_y = []

for label in label_names:
  text_files = os.listdir(os.path.join(data_dir,label))
  data_x.extend([os.path.join(data_dir,label,f) for f in text_files])
  data_y.extend([label for f in text_files])

# Load the text content
for idx, path in enumerate(data_x):
  # Load text
  with open(path) as file:
    data_x[idx] = file.read()

# Preview
print("data_x count:",len(data_x))
print("data_y count:",len(data_y))
print(data_x[:5])
print(data_y[:5])
print("Label counts:",np.unique(data_y, return_counts=True))

Number of classes: 3
Labels: ['health', 'politics', 'entertainment']
data_x count: 920
data_y count: 920
['WHO clarifies comments on asymptomatic spread of coronavirus: \'There\'s much unknown\'  (CNN)The World Health Organization tried on Tuesday to clear up confusing comments about how often people can spread the coronavirus when they do not have symptoms. The organization held a live Q&A on its social media pages to address questions about comments made by a WHO official that suggested asymptomatic people only rarely spread Covid-19.  The comments appeared to directly contradict guidance from public health organizations, including the US Centers for Disease Control and Prevention, which have said about a third of coronavirus infections may be asymptomatic. The CDC also estimates that 40% of coronavirus transmission is occurring before people feel sick, meaning they are presymptomatic. But it may boil down to how one defines "asymptomatic." \'There are so many unknowns\' Maria Van Ke

#### View Text

In [7]:
# Generate a random sample of index
data_samples = np.random.randint(0,high=len(data_x)-1, size=10)
for i,data_idx in enumerate(data_samples):
  print("Label:",data_y[data_idx],", Text:",data_x[data_idx])

Label: health , Text: National Coming Out Day: Coming out during a pandemic   (CNN)Erica Woodland first came out about his queer identity in middle school. In the last few years, he came out as transgender, and introduced his family to his new pronouns. Whether we come to understand our sexuality personally, with our family or with our broader community, coming out is a process, said Woodland, a clinical psychologist and founder of the National Queer and Trans Therapists of Color Network. October 11 is National Coming Out Day in the United States, celebrated each year to mark the  anniversary of the 1987 National March on Washington for Lesbian and Gay Rights.  After generations of progress in the LGBTQ community, marked by milestones such as the Stonewall Riots 51 years ago and the US Supreme Court decision for marriage equality in 2015, coming out is more accepted than it used to be. And it doesn't just happen once.  "Coming out is not one event. I experience this in my own coming ou

## Questions:

For this exercise you will use a transformer model **BERT** that was discussed in lecture. [Hugging Face](https://huggingface.co/) is an NLP-focused company with a large open-source library especially around the Transformers. They provide the transformers library that exposes an API to use many well-known transformer architectures, such as BERT, RoBERTa, GPT-2 or DistilBERT, that obtain state-of-the-art results on a variety of NLP tasks like text classification, information extraction, question answering, and text generation. These architectures come pre-trained with several sets of weights. Getting started with Transformers only requires to install the `pip install transformers`

You will use BERT direclty from the transformers package


## Question 1 : Build a text classification model using BERT (3.0 Points)

#### a) Prepare data for BERT

BERT requires the data to be tokenized in a specific way, for this you need to use the `BertTokenizer` from the `transformers` package from Hugging Face. Steps to prepare your dataset:

- Split data to train/validation
- Use `BertTokenizer` to tokenize the input text
- [BertTokenizer](https://huggingface.co/transformers/model_doc/bert.html#berttokenizer), use `bert-base-uncased` as the `vocab_file` argument
- When using `tokenizer.encode_plus(...)` use the `max_length=256` or some value `<=512`. You may run into OOM error during training if the value is high
- The output tokens from `tokenizer.encode_plus(...)` is a dictionary with the keys `'input_ids', 'token_type_ids', 'attention_mask'`
- Remember to convert the data y values `to_categorical`
- Create TF Datasets using the tokenized results. When using `tf.data.Dataset.from_tensor_slices(...,...)` look out for the x values passed in. `BERT` requires 3 inputs `'input_ids', 'token_type_ids', 'attention_mask'` as a tuple.
- Remember to apply `shuffle(..)` `batch(...)` `prefetch(..)` to your train and validation data

In [8]:
# Examples on how to use the BertTokenizer
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)

# Tokenizer encode_plus
text = "What you need to know about using them safely amid the pandemic"
outputs = tokenizer.encode_plus(text, 
                  add_special_tokens = True, # add [CLS], [SEP]
                  max_length = 15, # max length of the text that can go to BERT (<=512)
                  padding='max_length',
                  return_attention_mask = True, # add attention mask to not focus on pad tokens
                  truncation='longest_first',
                  return_tensors="tf"
              )
print("Tokenizer Output:",outputs)

# Tokenizer batch_encode_plus
text = ["What you need to know about using them safely amid the pandemic", 
        "A third of Medicare enrollees with coronavirus ended up in the hospital"]
outputs = tokenizer.batch_encode_plus(
        text,
        return_tensors='tf',
        add_special_tokens = True, # add [CLS], [SEP]
        return_token_type_ids=True,
        padding='max_length',
        max_length=15, # max length of the text that can go to BERT (<=512)
        return_attention_mask = True,
        truncation='longest_first'
    )
print("Tokenizer Output:",outputs)
print("Tokenizer Output Keys:", outputs.keys())


Tokenizer Output: {'input_ids': <tf.Tensor: shape=(1, 15), dtype=int32, numpy=
array([[  101,  2054,  2017,  2342,  2000,  2113,  2055,  2478,  2068,
         9689, 13463,  1996,  6090,  3207,   102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 15), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 15), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}
Tokenizer Output: {'input_ids': <tf.Tensor: shape=(2, 15), dtype=int32, numpy=
array([[  101,  2054,  2017,  2342,  2000,  2113,  2055,  2478,  2068,
         9689, 13463,  1996,  6090,  3207,   102],
       [  101,  1037,  2353,  1997, 27615, 25612, 10285,  2007, 21887,
        23350,  3092,  2039,  1999,  1996,   102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2, 15), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], d

In [99]:
# Datatset Params
batch_size = 8 # You can try higher values but may run into OOM errors depending on which GPU you are using
train_shuffle_buffer_size = 800
validation_shuffle_buffer_size = 200

def process_labels(labels: list, label2index: dict,  num_classes: int):
  label_ids = [label2index[label] for label in labels]
  return to_categorical(label_ids, num_classes=num_classes, dtype='int32')

label_names = os.listdir(data_dir)
num_classes = len(label_names) 

label2index = dict((name, index) for index, name in enumerate(label_names))
index2label = dict((index, name) for index, name in enumerate(label_names))

# Convert all y labels to categories
data_processed_y = process_labels(data_y, label2index, num_classes)

train_x, validate_x, train_y, validate_y = \
  train_test_split(data_x, data_processed_y, test_size=0.2)


# print(len(data_x), len(data_y))
ds_train = tf.data.Dataset.zip((
    tf.data.Dataset.from_tensor_slices(train_x),
    tf.data.Dataset.from_tensor_slices(train_y)
))
ds_validation = tf.data.Dataset.zip((
    tf.data.Dataset.from_tensor_slices(validate_x),
    tf.data.Dataset.from_tensor_slices(validate_y)
))


def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
  return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, label

def convert_example_to_feature(text):  
  return tokenizer.encode_plus(text, 
                               add_special_tokens = True, # add [CLS], [SEP]
                               max_length = 15, # max length of the text that can go to BERT (<=512)
                               padding='max_length',
                               return_attention_mask = True, # add attention mask to not focus on pad tokens
                               truncation='longest_first',
                               return_tensors="tf")

def encode_examples(ds, limit=-1):
  # prepare list, so that we can build up final TensorFlow dataset from slices.
  input_ids_list = []
  token_type_ids_list = []
  attention_mask_list = []
  label_list = []

  if (limit > 0):
      ds = ds.take(limit)

  for text, label in ds:
    bert_input = convert_example_to_feature(text.numpy().decode())
    input_ids_list.append(bert_input['input_ids'][0])
    token_type_ids_list.append(bert_input['token_type_ids'][0])
    attention_mask_list.append(bert_input['attention_mask'][0])
    label_list.append([label])

  return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_example_to_dict)


train_data = encode_examples(ds_train).             \
                shuffle(train_shuffle_buffer_size). \
                batch(batch_size).                  \
                prefetch(buffer_size=100)

validation_data = encode_examples(ds_validation).            \
                    shuffle(validation_shuffle_buffer_size). \
                    batch(batch_size).                       \
                    prefetch(buffer_size=100)


print("train_data", train_data)
print("validation_data", validation_data)

train_data <PrefetchDataset shapes: ({input_ids: (None, 15), token_type_ids: (None, 15), attention_mask: (None, 15)}, (None, 1, 3)), types: ({input_ids: tf.int32, token_type_ids: tf.int32, attention_mask: tf.int32}, tf.int32)>
validation_data <PrefetchDataset shapes: ({input_ids: (None, 15), token_type_ids: (None, 15), attention_mask: (None, 15)}, (None, 1, 3)), types: ({input_ids: tf.int32, token_type_ids: tf.int32, attention_mask: tf.int32}, tf.int32)>


Your train and validation dataset should look something like this
```
print("train_data",train_data)
print("validation_data",validation_data)

train_data <PrefetchDataset shapes: ({input_ids: (None, 256), token_type_ids: (None, 256), attention_mask: (None, 256)}, (None, 3)), types: ({input_ids: tf.int32, token_type_ids: tf.int32, attention_mask: tf.int32}, tf.int32)>
validation_data <PrefetchDataset shapes: ({input_ids: (None, 256), token_type_ids: (None, 256), attention_mask: (None, 256)}, (None, 3)), types: ({input_ids: tf.int32, token_type_ids: tf.int32, attention_mask: tf.int32}, tf.int32)>
```

#### b) BERT for Sequence Classification

- Build a model using `TFBertForSequenceClassification` from the `transformers` package from Hugging Face
- Load the pre-trained weights using `bert-base-uncased` make sure to pass the argument `num_labels`
- Train your model
- Ensure there is a plot of your training history

In [ ]:
############################
# Training Params
############################
learning_rate = 2e-5 # Try 5e-5, 3e-5, 2e-5
epochs = 5

# Free up memory
K.clear_session()

# Build BERT model
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

# Optimizer
optimizer = optimizers.Adam(lr=learning_rate, epsilon=1e-08)

# Loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile

metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[metric])

# Train model
bert_history = model.fit(train_data, epochs=epochs, validation_data=validation_data)

# Evaluate Model

#### c) Classification Results

- What was your validation accuracy?
- It should be more that 95%

the accuracy is around 96%, but i have poked around a few places to try to improve the accuracy, but it seems that the it has been limited by the size of the matrix. 

If I were to increase the size of the model to a more advanced or newer model, such as RoBERT or XLnet, I can probably better accurancy. 


---
## Question 2 : Conceptual (2.0 Points)


#### a) How does the encoder-decoder structure work for language modelling?


*Your answer here*

Encoder-decoder architecture has 2 components.

The first component is an encoder: it takes a variable-length sequence as the input and transforms it into a state with a fixed shape. The second component is a decoder: it maps the encoded state of a fixed shape to a variable-length sequence. 

The encoder-decoder architecture can handle inputs and outputs that are both variable-length sequences, thus is suitable for sequence transduction problems such as machine translation.


#### b) Explain in your own words what is the attention mechanism. Why do State of the art models use this concept?

*Your answer here*

The basic idea of attention is this: each time the model predicts an output word, it only uses parts of an input where the most relevant information is concentrated instead of an entire sentence. In other words, it only pays attention to some input words. 

#### c) What is the biggest benefit of transformers compared to seqtoseq models?

*Your answer here*

the answers lies in "Attention Is All You Need". Self-attention provides a powerful mechanism to train the model by the context of the words and label a word by the words that is preceding it and after it. 

Transformer architecture solves the limitation casued by seq2seq's sequential nature and enables long-range dependencies, so that a word can be directly correlated to words that are quite far away from it. 

Transformer eschews conventional network constructs while still outperforming existing systems.

#### d) Explain in your own words what the positional encoder is and why it is needed?

*Your answer here*

As each word in a sentence simultaneously flows through the Transformer’s encoder/decoder stack, The model itself doesn’t have any sense of position/order for each word. Consequently, there’s still the need for a way to incorporate the order of the words into our model.

This is where positional encoding comes in. it provides a vector with a clear sin and cos value to indicate the position of the words.